In [15]:
# https://adventofcode.com/2018/day/13

data = hcat(open("13.txt", "r") do file
    readlines(file) .|> line -> [convert(Int, c) for c in line]
end...)'

const CARTU, CARTD, CARTL, CARTR = convert.(Int, collect("^v<>"))
const TRAKV, TRAKH, TRAKU, TRAKD, TRAKX = convert.(Int, collect("|-/\\+"))

function tick(cart)
    (p, o, c) = cart
    y, x = Tuple(p)
    t = track[p]
    if t == TRAKU
        if o == CARTU
            o = CARTR
        elseif o == CARTD
            o = CARTL
        elseif o == CARTL
            o = CARTD
        elseif o == CARTR
            o = CARTU
        end
    elseif t == TRAKD
        if o == CARTU
            o = CARTL
        elseif o == CARTD
            o = CARTR
        elseif o == CARTL
            o = CARTU
        elseif o == CARTR
            o = CARTD
        end
    elseif t == TRAKX
        if o == CARTU && c == 0
            o = CARTL
        elseif o == CARTU && c == 2
            o = CARTR
        elseif o == CARTD && c == 0
            o = CARTR
        elseif o == CARTD && c == 2
            o = CARTL
        elseif o == CARTL && c == 0
            o = CARTD
        elseif o == CARTL && c == 2
            o = CARTU
        elseif o == CARTR && c == 0
            o = CARTU
        elseif o == CARTR && c == 2
            o = CARTD
        end
        c = (c + 1) % 3
    end

    if o == CARTU
        y -= 1
    elseif o == CARTD
        y += 1
    elseif o == CARTL
        x -= 1
    elseif o == CARTR
        x += 1
    end
    
    return (CartesianIndex(y, x), o, c)
end


tick (generic function with 1 method)

In [16]:
# part 1

track = copy(data)

# convert the carts on the track into positions, orientations, and turn counters
carts = [(p, track[p], 0) 
         for p in findall(t -> t in [CARTU, CARTD, CARTL, CARTR], track)]

# remove the carts from the track
track[[p for (p, o, c) in carts]] .= TRAKH

while true
    # find the first crash
    carts = [tick(c) for c in sort(carts)]
    crashes = [c1 for c1 in carts, c2 in carts
               if c1 != c2 && c1[1] == c2[1]]
    if !isempty(crashes)
        y, x = Tuple(crashes[1][1]) .- 1
        println("$x,$y")
        break
    end
end


80,100


In [26]:
# part 2

const WRECKED = (CartesianIndex(0, 0), 0, 0)

track = copy(data)

# convert the carts on the track into positions, orientations, and turn counters
carts = [(p, track[p], 0) 
         for p in findall(t -> t in [CARTU, CARTD, CARTL, CARTR], track)]

# remove the carts from the track
track[[p for (p, o, c) in carts]] .= TRAKH

while true
    # find and remove all crashes
    # . we must step through each cart, since a crash is removed
    #   instantly from the track
    # . mark crashed carts as WRECKED, so that we don't modify
    #   the cart list while iterating
    for i in 1:length(carts)
        if carts[i] != WRECKED
            carts[i] = tick(carts[i])
            crash = [j for (j, c) in enumerate(carts) 
                     if i != j && carts[i][1] == carts[j][1]]
            if !isempty(crash)
                j = crash[1]
                carts[i] = carts[j] = WRECKED
            end
        end
    end
    
    # stop when one cart remains
    carts = sort([c for c in carts if c != WRECKED])
    if length(carts) == 1
        y, x = Tuple(carts[1][1]) .- 1
        println("$x,$y ")
        break
    end
end


16,99 
